In [2]:
## Importing libraries 
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
import numpy as np
import soundfile as sf
from pathlib import Path
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-07-12 17:14:54 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-12 17:14:54 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [4]:
train_speaker_id = 91 ## TO BE CHANGED ## 
val_speaker_id = 44 ## TO BE CHANGED ## 
data_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE'
script_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/SCRIPT/'
train_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S' + str(train_speaker_id) + '.json'
val_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S' + str(val_speaker_id) + '.json'
#train_speakers = ['0091', '0124', '0135', '0148', '0173', '0178', '0190', '0194', '0208', '0218', '0245', '0276', '0289', '0308', '0314', '0317', '0329', '0330', '0337', '0349', '0379', '0397', '0432', '0443', '0451', '0462', '0488', '0489', '0490', '0501', '0537', '0543', '0557', '0591', '0592', '0603', '0620', '0627', '0634', '0658', '0659', '0675', '0743', '0762', '0763', '0766', '0778', '0780', '0785', '0790', '0791', '0792', '0798', '0799', '0812', '0831', '0872', '0881', '1058', '1064', '1444']
#val_speakers = ['0044', '0047', '0056', '0066', '0074']
train_speakers = ['0091'] ## TO BE CHANGED ## 
val_speakers = ['0044'] ## TO BE CHANGED ## 

## Function to build a manifest
def build_manifest(manifest_path, speakers):
  count = 0
  with open(manifest_path, 'w') as fout: 
    for i in speakers: 
      path_unzipped = data_dir + '/SPEAKER' + i
      if not os.path.exists(path_unzipped):
        path = data_dir + '/SPEAKER' + i + '.zip'
        zip_ref = zipfile.ZipFile(path, 'r')
        zip_ref.extractall(path=data_dir)
      n = 0
      wav_folder = data_dir + '/SPEAKER' + i + '/SESSION'
      while os.path.exists(wav_folder + str(n)):
        files = []
        for filename in os.listdir(wav_folder + str(n)):
          '''
          audio_path = os.path.join(wav_folder + str(n), filename)
          y, sr = torchaudio.load(audio_path)
          y = y.mean(dim=0) # if there are multiple channels, average them to single channel
          if sr != 22050:
            resampler = torchaudio.transforms.Resample(sr, 22050)
            y_resampled = resampler(y)
          torchaudio.save(audio_path[:-4] + '_resampled.wav', y.unsqueeze(0), sr)

          rate, data = wavfile.read(audio_path[:-4] + '_resampled.wav')
          # perform noise reduction
          reduced_noise = nr.reduce_noise(y=data, sr=rate)
          wavfile.write(audio_path[:-4] + '_clean.wav', rate, reduced_noise)
          files.append(filename[:-4] + '_clean.wav')
          '''
          files.append(filename)
        files = sorted(files)
        script_path = script_dir + filename[:6] + '.TXT'
        with open(script_path, encoding = 'utf-8') as script:
          for filename in files:
            #getting transcript 
            script.readline()
            line = script.readline().strip()
            words = line.split()
            edited_words = []
            for word in words:
              if (word[-1] != '>') and (word != '**'):
                edited_words.append(word.lower())
            transcript_line = ""
            for word in edited_words: 
              transcript_line = transcript_line + word + " "
            transcript_line = transcript_line[:-1]
            #getting audio file
            audio_path = os.path.join(wav_folder + str(n), filename)
            duration = librosa.core.get_duration(filename=audio_path)
            # Write the metadata to the manifest
            metadata = {
                "audio_filepath": audio_path,
                "text": transcript_line,
                "duration": duration
            }
            json.dump(metadata, fout)
            fout.write('\n')
        n = n+1
    
for s in train_speakers:
    train_speaker_id = int(s)
    print(train_speaker_id)
    train_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S' + str(train_speaker_id) + '.json'
    train_speakers = [] 
    train_speakers.append(s)
    build_manifest(train_manifest_path, train_speakers)
#build_manifest(val_manifest_path, val_speakers)

91


In [2]:
#home_path = '/home/dgxuser/Desktop/kailin'
home_path = '/home/dgxuser'
print(home_path)

/home/dgxuser


In [3]:
import os
import json

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt

from nemo.collections.tts.models import FastPitchModel
FastPitchModel.from_pretrained("tts_en_fastpitch")

from pathlib import Path
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_en_fastpitch_align.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_en_fastpitch_align.nemo").write_bytes(nemo_files[0].read_bytes())

[NeMo W 2022-06-29 17:48:16 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-29 17:48:17 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[NeMo I 2022-06-29 17:48:17 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
[NeMo I 2022-06-29 17:48:17 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo
[NeMo I 2022-06-29 17:48:17 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-06-29 17:48:21 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-06-29 17:48:47 experimental:27] Module <class 'nemo.collections.tts.torch.g2ps.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-06-29 17:48:48 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-06-29 17:48:48 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matri

[NeMo I 2022-06-29 17:48:48 features:200] PADDING: 1
[NeMo I 2022-06-29 17:48:51 save_restore_connector:243] Model FastPitchModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo.
Copying /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_en_fastpitch_align/26d7e09971f1d611e24df90c7a9d9b38/tts_en_fastpitch_align.nemo to ./


187023360

In [4]:
BRANCH = 'main'

!wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/fastpitch_finetune.py

!mkdir -p conf \
&& cd conf \
&& wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/fastpitch_align_v1.05.yaml \
&& cd ..

--2022-06-29 17:48:52--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/fastpitch_finetune.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1751 (1.7K) [text/plain]
Saving to: ‘fastpitch_finetune.py.2’

fastpitch_finetune. 100%[===================>]   1.71K  --.-KB/s    in 0s      

2022-06-29 17:48:53 (16.1 MB/s) - ‘fastpitch_finetune.py.2’ saved [1751/1751]

--2022-06-29 17:48:53--  https://raw.githubusercontent.com/nvidia/NeMo/main/examples/tts/conf/fastpitch_align_v1.05.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 

In [5]:
# additional files
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tts_dataset_files/cmudict-0.7b_nv22.01 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tts_dataset_files/heteronyms-030921 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/text_normalization/en/data/whitelist/lj_speech.tsv \
&& cd ..

--2022-06-29 17:48:54--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/cmudict-0.7b_nv22.01
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3721964 (3.5M) [text/plain]
Saving to: ‘cmudict-0.7b_nv22.01.2’

cmudict-0.7b_nv22.0 100%[===================>]   3.55M   963KB/s    in 6.8s    

2022-06-29 17:49:01 (532 KB/s) - ‘cmudict-0.7b_nv22.01.2’ saved [3721964/3721964]

--2022-06-29 17:49:01--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tts_dataset_files/heteronyms-030921
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awai

In [34]:
def load_wav(audio_file, target_sr=None):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
        sample_rate = f.samplerate
        if target_sr is not None and target_sr != sample_rate:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
    return samples.transpose()

# Get records from the training manifest
records = []
train_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json'
with open(train_manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path('/home/dgxuser/Desktop/kailin/melSpect')
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
    # Again, our finetuned FastPitch model doesn't use multiple speakers,
    # but we keep the code to support it here for reference
    if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
        speaker = torch.tensor([r['speaker']]).to(device)
    else:
        speaker = None
    
    with torch.no_grad():
        if "normalized_text" in r:
            text = spec_model.parse(r["normalized_text"], normalize=False)
        else:
            text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/melSpect_train.json'
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [35]:
# Get records from the validation manifest
records = []
val_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_val.json'
with open(val_manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path('/home/dgxuser/Desktop/kailin/melSpectVal')
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
    # Again, our finetuned FastPitch model doesn't use multiple speakers,
    # but we keep the code to support it here for reference
    if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
        speaker = torch.tensor([r['speaker']]).to(device)
    else:
        speaker = None
    
    with torch.no_grad():
        if "normalized_text" in r:
            text = spec_model.parse(r["normalized_text"], normalize=False)
        else:
            text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/melSpect_val.json'
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

In [43]:
# For original audio training (3 lines to change)
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/melSpect_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/melSpect_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=./S91_no_mixing \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-06-30 10:22:34 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-30 10:22:36 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-06-30 10:22:36 exp_manager:287] Experiments will be logged at


Epoch 24: 100%|██████████| 27/27 [00:18<00:00,  1.46it/s, loss=4.18, v_num=2-36]Exception ignored in: <function _ConnectionBase.__del__ at 0x7f48a22a4af0>
Traceback (most recent call last):
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Epoch 24: 100%|██████████| 27/27 [00:19<00:00,  1.39it/s, loss=4.18, v_num=2-36]
                                                                                Epoch 24, global step 625: 'v_loss' reached 1.16289 (best 1.16289), saving model to '/home/dgxuser/Desktop/kailin/S91_no_mixing/FastPitch/2022-06-30_10-22-36/checkpoints/FastPitch--v_loss=1.1629-epoch=24.ckpt' as top 3
Epoch 39: 100%|██████████| 25/25 [00:12<00:00,  1.99it/s, loss=4.09, v_num=2-36]


In [44]:
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

[NeMo I 2022-06-30 10:36:18 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-06-30 10:36:18 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-06-30 10:36:18 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-06-30 10:36:21 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-06-30 10:36:21 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-06-30 10:36:21 features:200] PADDING: 0


[NeMo W 2022-06-30 10:36:21 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-06-30 10:36:21 features:200] PADDING: 0
[NeMo I 2022-06-30 10:36:23 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [45]:
def infer(spec_gen_model, vocoder_model, str_input):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=None)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [50]:
from pydub import AudioSegment

text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_val.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/audio_files_val_v2/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_val_manifest.json'

speaker_id = 91
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f:
    n = 0 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      '''
      # resample at 16kHz
      sound = am.from_file(file_dir, format='wav', frame_rate=22050)
      sound = sound.set_frame_rate(16000)
      file_dir = audio_dir + str(n) + '_resampled.wav'
      sound.export(file_dir, format='wav')

      rate, data = wavfile.read(file_dir)
      # perform noise reduction
      reduced_noise = nr.reduce_noise(y=data, sr=rate)
      file_dir = audio_dir + str(n) + '_clean.wav'
      wavfile.write(file_dir, rate, reduced_noise)
      '''

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

S91_no_mixing/FastPitch/2022-06-30_10-22-36/checkpoints/FastPitch--v_loss=1.1629-epoch=24-last.ckpt
[NeMo I 2022-06-30 10:39:56 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-06-30 10:40:26 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-06-30 10:40:26 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/melSpect_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_durat

[NeMo I 2022-06-30 10:40:26 features:200] PADDING: 1


[NeMo W 2022-06-30 10:40:40 tts_tokenizers:356] Text: [AY1 KER1AH0NTLIY0 HHAE1V FAO1R WIH1NDOW0Z OW1PAH0N AH1P… and AY1 don't NOW1 why.] contains unknown char/phoneme: […]. Original text: [I currently have four windows open up… and I don't know why.]. Symbol will be skipped.


In [11]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_train.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/audio_files_train/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_train_manifest.json'

speaker_id = 91
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f:
    n = 0 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      '''
      # resample at 16kHz
      sound = am.from_file(file_dir, format='wav', frame_rate=22050)
      sound = sound.set_frame_rate(16000)
      file_dir = audio_dir + str(n) + '_resampled.wav'
      sound.export(file_dir, format='wav')

      rate, data = wavfile.read(file_dir)
      # perform noise reduction
      reduced_noise = nr.reduce_noise(y=data, sr=rate)
      file_dir = audio_dir + str(n) + '_clean.wav'
      wavfile.write(file_dir, rate, reduced_noise)
      '''

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

S91_no_mixing/FastPitch/2022-06-27_10-52-30/checkpoints/FastPitch--v_loss=5.6238-epoch=24-last.ckpt
[NeMo I 2022-06-29 17:53:26 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-06-29 17:53:56 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-06-29 17:53:56 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_duratio

[NeMo I 2022-06-29 17:53:56 features:200] PADDING: 1


[NeMo W 2022-06-29 17:54:24 tts_tokenizers:356] Text: [SAH1MWAH2N AY1 NOW1 recently KAH0MBAY1ND MEY1PAH0L syrup & BAH1TER0D PAA1PKAO2RN THIH1NGKIH0NG it WUH1D TEY1ST LAY1K KEH1RAH0MAH0L PAA1PKAO2RN. it didn't and DHEY1 don't REH2KAH0MEH1ND anyone EH1LS do it either.] contains unknown char/phoneme: [&]. Original text: [Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would taste like caramel popcorn. It didn't and they don't recommend anyone else do it either.]. Symbol will be skipped.
[NeMo W 2022-06-29 17:54:26 tts_tokenizers:356] Text: [SAH1MWAH2N AY1 NOW1 recently KAH0MBAY1ND MEY1PAH0L syrup & BAH1TER0D PAA1PKAO2RN THIH1NGKIH0NG it WUH1D TEY1ST LAY1K KEH1RAH0MAH0L PAA1PKAO2RN. it didn't and DHEY1 don't REH2KAH0MEH1ND anyone EH1LS do it either.] contains unknown char/phoneme: [&]. Original text: [Someone I know recently combined Maple Syrup & buttered Popcorn thinking it would taste like caramel popcorn. It didn't and they don't recommend anyone else do it 

In [12]:
## Importing libraries 
!pip install jiwer
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2

import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [13]:
## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml --no-check-certificate

mkdir: cannot create directory ‘configs’: File exists
--2022-06-29 17:55:24--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/config.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4282 (4.2K) [text/plain]
Saving to: ‘configs/config.yaml.1’

config.yaml.1       100%[===================>]   4.18K  --.-KB/s    in 0s      

2022-06-29 17:55:25 (20.5 MB/s) - ‘configs/config.yaml.1’ saved [4282/4282]



In [14]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-06-29 17:55:25 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/versions/1.0.0rc1/files/stt_en_quartznet15x5.nemo to /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
100% [...................................................] 70993538 / 70993538[NeMo I 2022-06-29 17:55:39 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-06-29 17:55:40 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-06-29 17:55:40 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-06-29 17:55:40 features:200] PADDING: 16
[NeMo I 2022-06-29 17:55:41 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [51]:
import json 
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_val_manifest.json'
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [52]:
error = wer(transcript, predicted)
print('Original Word Error Rate: ' + str(error))

Original Word Error Rate: 0.38343108504398826
